In [1]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/recurrent_neural_network/main.py#L39-L58

In [2]:
!pip uninstall torch -y
!pip uninstall torchvision -y
!pip install torch 
!pip install torchvision
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

Found existing installation: torch 1.7.0
Uninstalling torch-1.7.0:
  Successfully uninstalled torch-1.7.0
Found existing installation: torchvision 0.8.1
Uninstalling torchvision-0.8.1:
  Successfully uninstalled torchvision-0.8.1
     |████████████████████████████████| 831.4 MB 1.7 kB/s  eta 0:00:01     |███████▉                        | 203.8 MB 56.1 MB/s eta 0:00:12     |█████████                       | 235.5 MB 43.4 MB/s eta 0:00:14     |██████████▉                     | 281.2 MB 69.5 MB/s eta 0:00:08     |████████████▏                   | 315.1 MB 69.5 MB/s eta 0:00:08     |█████████████████████           | 544.2 MB 63.5 MB/s eta 0:00:05     |██████████████████████████      | 675.9 MB 63.2 MB/s eta 0:00:03     |██████████████████████████▎     | 684.0 MB 63.2 MB/s eta 0:00:03     |███████████████████████████▋    | 717.9 MB 43.0 MB/s eta 0:00:03     |████████████████████████████▍   | 738.7 MB 63.7 MB/s eta 0:00:02     |█████████████████████████████   | 754.6 MB 63.7 MB/s eta 0:00:02

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Hyper-parameters
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



/opt/conda/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # https://adventuresinmachinelearning.com/keras-lstm-tutorial/
        # input_size – The number of expected features in the input x
        # hidden_size – The number of features in the hidden state 
        # num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1
        # https://adventuresinmachinelearning.com/keras-lstm-tutorial/
        # 
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [8]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5033
Epoch [1/2], Step [200/600], Loss: 0.3397
Epoch [1/2], Step [300/600], Loss: 0.1725
Epoch [1/2], Step [400/600], Loss: 0.0467
Epoch [1/2], Step [500/600], Loss: 0.0944
Epoch [1/2], Step [600/600], Loss: 0.0649
Epoch [2/2], Step [100/600], Loss: 0.1007
Epoch [2/2], Step [200/600], Loss: 0.0695
Epoch [2/2], Step [300/600], Loss: 0.1296
Epoch [2/2], Step [400/600], Loss: 0.1223
Epoch [2/2], Step [500/600], Loss: 0.0959
Epoch [2/2], Step [600/600], Loss: 0.0757


In [11]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')